In [19]:
!pip install seaborn

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/83/11/00d3c3dfc25ad54e731d91449895a79e4bf2384dc3ac01809010ba88f6d5/seaborn-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for pandas>=1.2 from https://files.pythonhosted.org/packages/71/00/6beaeeba7f075d15ea167a5caa039b861e58ff2f58a5b659abb9b544c8f6/pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata
  Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for matplotlib!=3.6.1,>=3.4 from https://files.pythonhosted.org/packages/50/ce/a6bc93f7a44dd1fd23698698e369e141f4f24e7098d0a5937808afee3f5e/matplotlib-3.8.3-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/8e/ae/a6353db548bff1a592b85ae6bb80275f0a51dc25a0410d059e5b33183e36/contourpy-1.2.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.p

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Python312\\share'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
!pip install torch torchvision torchaudio


  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/a1/e3/5fd1c7d1836141b0bd253502be910c73d84a34ced27a289ea4ca4975ef3b/torchvision-0.17.1-cp312-cp312-win_amd64.whl.metadata
  Using cached torchvision-0.17.1-cp312-cp312-win_amd64.whl.metadata (6.6 kB)
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/77/f7/6ddef43a933637ad0d80432420c6928882ca8bf19361e20bf8f7417228ba/torchaudio-2.2.1-cp312-cp312-win_amd64.whl.metadata
Using cached torchvision-0.17.1-cp312-cp312-win_amd64.whl (1.2 MB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
    --------------------------------------- 0.0/2.4 MB 393.8 kB/s eta 0:00:06
   - -------------------------------------- 0.1/2.4 MB 655.4 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/2.4 MB 919.0 kB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.4 


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import torch
import torch.nn as nn
from torch.nn.functional import softmax
from sklearn.metrics import accuracy_score, recall_score
from transformers import logging
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler, WeightedRandomSampler
logging.set_verbosity_error()


In [31]:
# Load dataset
df = pd.read_csv('JRA3500.csv')

In [32]:
# Remove null values
df.dropna(subset=['title', 'description', 'component'], inplace=True)


In [33]:
def preprocess_text(text):
    # Delete new line tags and other known tags
    text = re.sub(r'[\n\r\t]', ' ', text)
    
    # Delete URL
    text = re.sub(r'http\S+', '', text)
    
    # Delete stack trace
    text = re.sub(r'Stack trace:.*(\n.*){1,10}', '', text)
    
    # Delete hex code
    text = re.sub(r'0x[0-9a-fA-F]+', '', text)
    
    return text.strip()


In [34]:
# Apply preprocessing to the dataframe
df['title'] = df['title'].apply(preprocess_text)
df['description'] = df['description'].apply(preprocess_text)
# Combine the title and description into a single text field
df['text'] = df['title'] + " " + df['description']


In [35]:
# Get the count of each component
component_counts = df['component'].value_counts()

# Print the counts
print(component_counts)

component
Issue - Fields                                                 250
Documentation - All                                            238
Dashboards & Reports - Gadgets                                 225
Events & Notifications - Email                                 172
User Management - Others                                       168
                                                              ... 
JQL ,System Administration - Jelly                               1
Ecosystem - REST API,Issue - Comments                            1
Filtering & Indexing ,Project Administration - Workflows         1
Administration ,Issue Navigation & Search - Bulk Operations      1
Issue - Fields,Issue Navigation & Search - Export                1
Name: count, Length: 303, dtype: int64


In [36]:
component_counts = df['component'].value_counts()
df = df[~df['component'].isin(component_counts[component_counts < 100].index)]


In [37]:
# Get the count of each component
component_counts = df['component'].value_counts()

# Print the counts
print(component_counts)

component
Issue - Fields                              250
Documentation - All                         238
Dashboards & Reports - Gadgets              225
Events & Notifications - Email              172
User Management - Others                    168
Administration                              154
JQL                                         146
Plugins                                     145
Project Administration - Workflows          124
Infrastructure & Services - Installation    123
Issue navigator                             118
Issue - Attachments                         102
Name: count, dtype: int64


In [62]:
# Use the Roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Dataset Class
class ComponentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        
        # Encode the text into tensors and return a dictionary
        data = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': data['input_ids'].flatten(),
            'attention_mask': data['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }




In [63]:
# Encode the labels into unique integers
label_unique = df['component'].unique()
label_dict = {label: idx for idx, label in enumerate(label_unique)}
df['label'] = df['component'].map(label_dict)


In [64]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=17,
    stratify=df.label.values
)

df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_test, 'data_type'] = 'test'
MAX_LEN = 256

In [65]:
# Create the datasets
train_dataset = ComponentDataset(
    texts=df[df.data_type=='train'].text.values,
    labels=df[df.data_type=='train'].label.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

test_dataset = ComponentDataset(
    texts=df[df.data_type=='test'].text.values,
    labels=df[df.data_type=='test'].label.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)


In [66]:
# Create Data Loaders
BATCH_SIZE = 16

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=BATCH_SIZE
)

test_dataloader = DataLoader(
    test_dataset,
    sampler=SequentialSampler(test_dataset),
    batch_size=BATCH_SIZE
)


In [67]:
from transformers import RobertaForSequenceClassification, AdamW
from torch.optim.lr_scheduler import OneCycleLR

# Load the RoBERTa model for sequence classification
model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=len(label_dict), # The number of output labels
    output_attentions=False,
    output_hidden_states=False
)


In [68]:
# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-6, weight_decay=5e-6)


C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [73]:
# Define the scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader) * epochs
)


In [74]:
# Move the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [82]:
from sklearn.metrics import f1_score
import random
# Define training parameters
epochs = 3
max_grad_norm = 1.0
early_stopping_counter = 0
early_stopping_patience = 3
best_val_loss = float('inf')

# Set seed for reproducibility
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


In [83]:
from tqdm import tqdm
import time

print("Starting training...")
# Training loop
for epoch in range(1, epochs + 1):
    
    print(f"\nEpoch {epoch}/{epochs}")
    model.train()
    loss_train_total = 0

    # Initialize tqdm progress bar
    progress_bar = tqdm(train_dataloader, desc='Batch', leave=False, total=len(train_dataloader))
    
    for batch in progress_bar:
        model.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss = outputs.loss
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        # Update the progress bar with the loss information
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})
        
    # Calculate the average loss over all of the batches.
    train_loss = loss_train_total / len(train_dataloader)
    
    # Closing the progress bar
    progress_bar.close()

    print(f"Epoch {epoch}/{epochs} | Training loss: {train_loss:.4f}")


Starting training...

Epoch 1/3


Epoch 1/3 | Training loss: 2.4743

Epoch 2/3


Epoch 2/3 | Training loss: 2.3450

Epoch 3/3


Epoch 3/3 | Training loss: 2.0714


In [87]:
from sklearn.metrics import classification_report
import numpy as np

# Function to evaluate the model
def evaluate(model, dataloader):
    model.eval()
    
    predictions, true_labels = [], []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=1).tolist())
            true_labels.extend(batch['labels'].tolist())
    
    return true_labels, predictions

# Evaluate the model
true_labels, predictions = evaluate(model, test_dataloader)

# Compute the performance metrics
report = classification_report(true_labels, predictions, target_names=label_dict.keys())
print(report)


                                          precision    recall  f1-score   support

                User Management - Others       0.57      0.80      0.67        25
Infrastructure & Services - Installation       0.00      0.00      0.00        18
          Events & Notifications - Email       0.48      0.62      0.54        26
                        Issue navigator        0.00      0.00      0.00        18
                     Issue - Attachments       0.00      0.00      0.00        15
      Project Administration - Workflows       0.90      0.47      0.62        19
                     Documentation - All       0.32      0.69      0.43        36
                                Plugins        0.57      0.18      0.28        22
                         Administration        0.50      0.04      0.08        23
          Dashboards & Reports - Gadgets       0.69      0.59      0.63        34
                                    JQL        0.73      0.36      0.48        22
               

C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [88]:
# Saving the model
model_path = "roberta_component_classification"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# To load the model and tokenizer later
from transformers import RobertaTokenizer, RobertaForSequenceClassification
loaded_tokenizer = RobertaTokenizer.from_pretrained(model_path)
loaded_model = RobertaForSequenceClassification.from_pretrained(model_path)


In [89]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 44.41%
